1. Import Necessary Libraries

TensorFlow/Keras: For building and training the model.

zipfile: To extract the dataset from a ZIP archive.

os and shutil: For directory management.

sklearn.model_selection: Provides tools like train_test_split


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
import os
import zipfile
import shutil

2.SETTING UP DIRECTORIES

In [2]:
# Set up directories for training and validation data
train_dir = '/content/dataset'
val_dir = '/content/dataset'

3.EXTRACTING THE DATASET

In [3]:
# Step 1: Extract Dataset
zip_path = '/content/dataset.zip'  # Replace with your zip file path
extract_dir = 'dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

4.PREPROCESSING THE DATASET

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Train generator
train_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(224, 224),  # Resize images to 224x224 (compatible with VGG16 input size)
    batch_size=32,
    class_mode='binary',  # Binary classification (benign/malignant)
    subset='training'  # Use 80% of data for training
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use 20% of data for validation
)


Found 1050 images belonging to 1 classes.
Found 262 images belonging to 1 classes.


5.LOADING THE PRETRAINED VGG-19 MODEL

In [5]:
# Load the VGG-19 model with pretrained weights, excluding the top layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


6.UPDATING PRETRAINED LAYERS

In [6]:

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = True

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

7.ADDING CUSTOM LAYERS

In [8]:
# Add custom layers on top of VGG-19
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=x)


8.COMPILE THE MODEL

In [9]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


9.TRAIN THE MODEL

In [10]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # Adjust epochs based on requirements
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2741s 86s/step - accuracy: 0.8873 - loss: 0.1491 - val_accuracy: 1.0000 - val_loss: 2.5848e-31
Epoch 2/10
 1/32 ━━━━━━━━━━━━━━━━━━━━ 41:13 80s/step - accuracy: 1.0000 - loss: 5.2028e-30

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


32/32 ━━━━━━━━━━━━━━━━━━━━ 322s 8s/step - accuracy: 1.0000 - loss: 5.2028e-30 - val_accuracy: 1.0000 - val_loss: 1.1505e-31
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2782s 87s/step - accuracy: 1.0000 - loss: 8.2265e-27 - val_accuracy: 1.0000 - val_loss: 3.0476e-35
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 322s 8s/step - accuracy: 1.0000 - loss: 3.5196e-31 - val_accuracy: 1.0000 - val_loss: 3.0478e-35
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2782s 86s/step - accuracy: 1.0000 - loss: 2.3298e-28 - val_accuracy: 1.0000 - val_loss: 2.1475e-35
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 322s 8s/step - accuracy: 1.0000 - loss: 1.2695e-28 - val_accuracy: 1.0000 - val_loss: 2.1079e-35
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2781s 87s/step - accuracy: 1.0000 - loss: 1.5426e-28 - val_accuracy: 1.0000 - val_loss: 2.0240e-35
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 322s 8s/step - accuracy: 1.0000 - loss: 1.6570e-29 - val_accuracy: 1.0000 - val_loss: 2.1194e-35
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 2773s 85s/step - accur

10.EVALUATE THE MODEL

In [11]:
# Step 7: Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(val_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 194s 21s/step - accuracy: 1.0000 - loss: 2.2453e-35
Test Accuracy: 100.00%


In [12]:
from tensorflow.keras import layers,models

11.SAVE THE MODEL

In [13]:
model.save('vgg_19_model.h5')